In [1]:
import os.path as op
import os
import pandas as pd
import numpy as np
from copy import deepcopy

In [2]:
data_dirs = ['../data/results_D100'] # , '../data/results_D25']
files = [op.join(d, f)
         for d in data_dirs
         for f in os.listdir(d)
         if f.endswith('csv')]

In [3]:
full_results = []
for fl in files:
    tmpres = pd.read_csv(fl, index_col=0)
    fname_pieces = fl.split('_')
    dset = fname_pieces[1].split('/')[0]
    experiment = fname_pieces[2]
    n_mca = int(fname_pieces[3])
    clf = fname_pieces[-4]
    dimens = fname_pieces[-2]
    preproc = fname_pieces[-3]
    random_seed = fname_pieces[-1].strip('.csv')
    resdict = {
        "Classifier": clf,
        "Experiment": experiment,
        "Simulations": n_mca,
        "Preprocessing": preproc,
        "Dimensionality": dimens,
        "Dataset": dset,
        "rs": random_seed
    }

    for idx, r in tmpres.iterrows():
        resdict_copy = deepcopy(resdict)
        # Only add ref performance for the "full" dataset
        if r.aggregation == "ref" and n_mca != 20:
            continue
        for k, v in r.items():
            resdict_copy[k] = v
        full_results += [resdict_copy]
        del resdict_copy

df_results = pd.DataFrame.from_dict(full_results)

In [4]:
df_results

,Classifier,Experiment,Simulations,Preprocessing,Dimensionality,Dataset,rs,aggregation,acc,f1,p_acc,p_f1,test_acc,test_f1,p_test_acc,p_test_f1,test_mean_acc,test_mean_f1,n_models
0,svc,mca,14,loggraph,fa,D100,41,median,0.666667,0.769231,0.081800,0.076800,0.700000,0.823529,0.0230,0.00500,0.670000,0.795820,5
1,rfc,mca,16,graph,fa,D100,41,mega,0.755690,0.836674,0.000000,0.000000,0.711765,0.825623,0.0000,0.00000,0.735294,0.832746,5
2,lrc,mca,18,loggraph,fa,D100,41,mean,0.550833,0.644421,0.358800,0.340000,0.600000,0.692308,0.1310,0.15900,0.630000,0.721086,5
3,ada,mca,10,zgraph,pca,D100,41,none,0.767217,0.823213,0.088412,0.088552,0.701000,0.800552,0.0207,0.02572,0.690800,0.772984,250
4,ada,mca,10,zgraph,pca,D100,41,meta,0.769167,0.830866,0.051200,0.053800,0.705000,0.803871,0.0195,0.02450,0.709000,0.788887,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,knn,mca,2,rankgraph,pca,D100,41,none,0.718833,0.826089,0.026000,0.017220,0.650000,0.787879,0.0600,0.02700,0.668000,0.798788,50
2196,knn,mca,2,rankgraph,pca,D100,41,meta,0.679167,0.787066,0.054000,0.046400,0.650000,0.787879,0.0600,0.02700,0.660000,0.776502,5
2197,knn,mca,18,zgraph,pca,D100,41,mean,0.719167,0.801591,0.069200,0.076200,0.750000,0.838710,0.0080,0.01000,0.740000,0.828341,5
2198,lrc,mca,20,graph,fa,D100,41,mean,0.718333,0.773141,0.059000,0.092400,0.700000,0.785714,0.0180,0.02100,0.660000,0.744184,5


In [5]:
df_results.to_csv("../data/aggregation_results.csv", index=False)